In [3]:
!pip3 install -r ../requirements.txt

  Using cached dash-2.17.0-py3-none-any.whl.metadata (10 kB)
  Using cached numpy-1.26.4-cp311-cp311-macosx_10_9_x86_64.whl.metadata (61 kB)
  Using cached pandas-2.2.2-cp311-cp311-macosx_10_9_x86_64.whl.metadata (19 kB)
  Using cached plotly-5.22.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached scipy-1.13.0-cp311-cp311-macosx_10_9_x86_64.whl.metadata (60 kB)
  Using cached gunicorn-22.0.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached flask-3.0.3-py3-none-any.whl.metadata (3.2 kB)
  Using cached werkzeug-3.0.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached dash_html_components-2.0.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached dash_core_components-2.0.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached dash_table-5.0.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached importlib_metadata-7.1.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached retrying-1.3.4-py3-none-any.whl.metadata (6.9 kB)
  Using cached 

In [4]:
import pandas as pd
import numpy as np
import datetime as dt

In [5]:
X = pd.read_csv('../assets/data.csv')

In [6]:
def pivotToClose(df:pd.DataFrame) -> pd.DataFrame:
    if not pd.api.types.is_datetime64_any_dtype(df['DATE']):
        df['DATE'] = pd.to_datetime(df['DATE'])

    # Pivot the DataFrame to wide format
    transformed_df = df.pivot(index='DATE', columns='ID', values='CLOSE')

    return transformed_df

def getData(df:pd.DataFrame, stocks:list[str], start:pd.DatetimeIndex, end:pd.DatetimeIndex) -> tuple[pd.DataFrame, pd.DataFrame]:
    X = df.loc[df.ID.isin(stocks)].loc[df.DATE > start].loc[df.DATE < end]
    stockData = X[['ID','DATE','CLOSE']]
    stockData = pivotToClose(stockData)
    returns = stockData.pct_change()
    meanReturns = returns.mean()
    covMatrix = returns.cov()
    return returns, meanReturns, covMatrix

In [8]:
Xp = pivotToClose(X)
Xp

ID,AAPL,AMT,AMZN,COST,GE,GLD,JNJ,JPM,MARK,NFLX,NVDA,PFE,SO,T,TSLA,XOM
DATE,,,,,,,,,,,,,,,,
2010-06-02,NaN,41.45,6.316,58.950,78.2486,119.7800,59.73,39.55,21.5000,15.826,3.1800,14.4088,32.69,18.7033,NaN,60.770
2010-06-03,NaN,42.48,6.438,57.930,78.7272,117.9600,59.77,39.10,21.0000,15.979,3.1750,14.4396,32.83,18.7184,NaN,61.560
2010-06-04,NaN,41.51,6.139,56.173,75.1857,119.1900,58.01,37.62,23.7000,15.683,3.0238,13.9870,32.13,18.2429,NaN,59.525
2010-06-07,NaN,41.90,6.101,55.850,73.7499,121.4900,58.00,36.72,18.9000,15.594,2.8756,13.7642,32.28,18.3562,NaN,59.300
2010-06-08,NaN,42.25,5.942,56.660,74.0849,121.0025,58.64,37.78,19.5000,15.761,2.7950,13.7927,32.84,18.8468,NaN,61.240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-29,173.50,174.99,180.960,726.330,164.4900,216.1800,146.82,193.28,0.1279,559.490,877.5700,25.6400,74.39,17.0200,194.05,119.640
2024-04-30,170.33,171.56,175.000,722.900,161.8200,211.8700,144.59,191.74,0.1374,550.640,864.0200,25.6200,73.50,16.8900,183.28,118.270
2024-05-01,169.30,176.84,179.000,722.220,159.7000,213.7900,151.18,191.86,0.1250,551.710,830.4100,27.1800,74.52,16.9200,179.99,116.030


In [10]:
stocks = ['AAPL','NVDA','JNJ']
startDate = '2015-01-01'
endDate = '2017-01-01'

R, M, C = getData(X,stocks,startDate,endDate)

In [13]:
C

ID,AAPL,JNJ,NVDA
ID,,,
AAPL,0.000250,0.000056,0.000134
JNJ,0.000056,0.000091,0.000059
NVDA,0.000134,0.000059,0.000677


In [14]:
Xp.corr()

ID,AAPL,AMT,AMZN,COST,GE,GLD,JNJ,JPM,MARK,NFLX,NVDA,PFE,SO,T,TSLA,XOM
ID,,,,,,,,,,,,,,,,
AAPL,1.000000,0.818438,0.895913,0.974915,-0.442202,0.721032,0.841508,0.876631,-0.668922,0.811180,0.835031,0.656944,0.915893,-0.750538,0.940605,0.047718
AMT,0.818438,1.000000,0.931609,0.823598,-0.624366,0.504516,0.919432,0.873072,-0.623825,0.886309,0.534176,0.817372,0.848621,-0.400717,0.762486,-0.294602
AMZN,0.895913,0.931609,1.000000,0.892845,-0.560846,0.614398,0.886273,0.924299,-0.611900,0.963709,0.723079,0.701087,0.824049,-0.550323,0.836457,-0.253707
COST,0.974915,0.823598,0.892845,1.000000,-0.390919,0.684000,0.853897,0.902803,-0.684916,0.820403,0.879447,0.668768,0.923873,-0.672717,0.883961,0.122843
GE,-0.442202,-0.624366,-0.560846,-0.390919,1.000000,-0.430086,-0.424043,-0.416768,0.431406,-0.536170,-0.125125,-0.422265,-0.378551,0.496005,-0.404123,0.487281
GLD,0.721032,0.504516,0.614398,0.684000,-0.430086,1.000000,0.383951,0.491001,-0.609163,0.519312,0.678353,0.191203,0.652065,-0.662470,0.680331,0.000419
JNJ,0.841508,0.919432,0.886273,0.853897,-0.424043,0.383951,1.000000,0.916143,-0.528764,0.842103,0.602714,0.874430,0.860031,-0.365647,0.765212,-0.038450
JPM,0.876631,0.873072,0.924299,0.902803,-0.416768,0.491001,0.916143,1.000000,-0.565728,0.919836,0.768984,0.728025,0.838737,-0.474015,0.779088,-0.017900
MARK,-0.668922,-0.623825,-0.611900,-0.684916,0.431406,-0.609163,-0.528764,-0.565728,1.000000,-0.541978,-0.540075,-0.449640,-0.705817,0.489405,-0.575146,0.026384
